# Fixe-Glisse (ou Stick-Slip) - Modélisation d'une corde de violon
Activité numérique mettant en oeuvre la capacité numérique suivante du programme de MP :
><i>À l'aide d'un langage de programmation, simuler une situation mécanique dans laquelle intervient au moins un changement de mode de glissement.</i>

<h2><span class="tocSkip"> Plan de l'activité :</span></h2>

<div class="toc-item"><ul class="toc-item">
    <li><span><a href="#Présentation-du-Stick-slip-:-cas-du-mouvement-d'une-corde-de-violon" data-toc-modified-id="Présentation-du-Stick-slip-1.1">Présentation du Stick-slip : cas du mouvement d'une corde de violon</a></span>
    <li><span><a href="#Modélisation" data-toc-modified-id="Modélisation-1.2">Modélisation</a></span></li>
    <li><span><a href="#Résolution-numérique" data-toc-modified-id="Résolution-numérique-1.3">Résolution numérique</a></span>
       <ul class="toc-item">
         <li><span><a href="#Conditions-initiales" data-toc-modified-id="Conditions-initiales-1.3.1">Conditions initiales</a></span></li>
         <li><span><a href="#Détermination-de-l'état-du-système" data-toc-modified-id="Détermination-de-l'état-du-système-1.3.2">Détermination de l'état du système</a></span></li>
         <li><span><a href="#Résolution-en-cas-d'adhérence" data-toc-modified-id="Résolution-en-cas-d'adhérence-1.3.3">Résolution en cas d'adhérence</a></span></li>
         <li><span><a href="#Résolution-en-cas-de-glissement" data-toc-modified-id="Résolution-en-cas-de-glissement-1.3.4">Résolution en cas de glissement</a></span></li>
         <li><span><a href="#Résolution-entière" data-toc-modified-id="Résolution-entière-1.3.5">Résolution entière</a></span></li>
         <li><span><a href="#Analyse-des-résultats" data-toc-modified-id="Analyse-des-résultats-1.3.6">Analyse des résultats</a></span></li></ul></li>
    <li><span><a href="#Annexe" data-toc-modified-id="Annexe-1.4">Annexe</a></span></li>
    <li><span><a href="#Bibliographie" data-toc-modified-id="Bibliographie-1.5">Bibliographie</a></span></li></ul></div>

## Présentation du Stick-slip : cas du mouvement d'une corde de violon


Le mouvement d'une corde de violon déplacée par un archet fait apparaître une alternance de phase d'adhérence et de glissement ("stick-slip"), voir la vidéo ci-dessous:

<a href=https://raw.githubusercontent.com/epertreux/CN2_Stick-slip/main/Stick-slip.mp4><img src="https://raw.githubusercontent.com/epertreux/CN2_Stick-slip/main/video.png" width="400" height="200"></a>


Nous allons étudier une situation expérimentale qui modélise cette situation :

<img src="https://raw.githubusercontent.com/epertreux/CN2_Stick-slip/main/situation.png" width="400" height="200">


Le solide S1 est entraîné à une vitesse constante $\vec{v}_0$ par un moteur. Le solide S2 frotte sur S1 et est retenu à gauche par un dynamomètre qui est ici représenté par un ressort de raideur $k$ et de longueur à vide $l_0$.

On appelle $f_s$ le coefficient de frottement statique entre les deux solides, et $f_d$ le coefficient de frottement dynamique. La masse de S2 est notée $m$.

Ce mouvement de stick-slip peut aussi modéliser le crissement d'une craie sur un tableau (ou d'une chaise sur le sol). La hauteur du son émis est alors directement liée à la fréquence du phénomène de stick-slip.

*L'objectif de cette activité numérique est de comprendre la situation étudiée et de prédire la fréquence du phénomène de stick-slip en fonction des différents paramètres de l'expérience.*




## Modélisation

**Question 1**
> Dans la modélisation, quel solide (S1 ou S2) joue le rôle de la corde ? de l'archet ? 
> Qui joue le rôle du ressort dans la situation réelle ?
> Quelle est la différence entre la situation réelle de la vidéo et la situation modélisée ?

**Question 2**
> Représenter sur le schéma les différentes forces s'appliquant sur S2.

**Question 3**
> Lorsque S1 est en situation d'adhérence, que vaut $v= \frac{dx}{dt}$ ? A quelle condition sur $x$ sort-on de l'adhérence et rentre-t-on en glissement ?

**Question 4**
> Lorsque S1 est en situation de glissement, quelle est l'équation différentielle vérifiée par $x$ ? A quelle condition sur $v=\frac{dx}{dt}$ (que l'on exprimera d'abord sous forme d'une égalité, puis d'une inégalité) va-t-on sortir du glissement ?

## Résolution numérique

On s'aperçoit ainsi que selon l'état (adhérence ou glissement) du système, l'équation à résoudre pour déterminer l'évolution de $x(t)$ diffère : comme ces états vont se succéder, il faudra à chaque fois résoudre les équations, en prenant en compte les continuités de $x$ et $\frac{dx}{dt}$ pour raccorder les solutions. Ce travail, s'il est possible, est toutefois laborieux... et la résolution numérique va nous permettre ici de visualiser rapidement le mouvement du solide $S1$.

### Conditions initiales

Dans un premier temps, on va exécuter le code ci-dessous, qui importe les modules Python nécessaires. Les valeurs numériques des différents paramètres du problème sont rentrées dans les unités du système international.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

v0 = 0.1 
fs = 0.2
fd = 0.1

k = 20 # constante de raideur du ressort

m = 1  # kg
g = 9.81  # accérlération de pesanteur


t = [0]     # t correspond à la liste des instants de calcul de la position x et de la vitesse xp (pour "xpoint")
tf = 20     # instant final de l'étude
dt = 0.001  # pas de calcul

**Question 5**
> Rappeler le compromis à faire sur le choix de $dt$ pour qu'il soit adapté à la résolution numérique.

On a ensuite besoin d'une variable ```etat``` qui indiquera l'état dans lequel se situe S2 :
- l'état d'adhérence sera caractérisé par ```etat=0```
- l'état de glissement sera caractérisé par ```etat=1```

Par ailleurs, on va créer deux listes : ```l_x```et ```l_xp``` correspondant aux valeurs de $x$ et $\frac{dx}{dt}$ évalués aux instants indiqués dans la liste ```t```. On aura donc :
$$\mbox{l_x[i]}=x\left(t=t[i]\right)$$
$$\mbox{l_xp[i]}=\frac{dx}{dt}(t=t[i])$$

**Question 6**
> Compléter le code suivant pour définir l'état initial, correspondant au solide S1, placé sur le tapis, adhérent, en $x=0$.

In [ ]:
etat = ###   

l_x = [0]
l_xp = ####  

### Détermination de l'état du système

Le problème à résoudre est donc :

<img src="https://raw.githubusercontent.com/epertreux/CN2_Stick-slip/main/algo2.png" width="500" height="250">

Ainsi, à chaque nouvelle estimation de $x$ et $\frac{dx}{dt}$, il faut vérifier si le système reste dans le même état ou s'il change d'état.

On définit alors la fonction ```state(x,xp,etat)``` qui retourne la valeur actualisée de ```etat``` à partir de la position ```x``` et de la vitesse ```xp```, nouvellement calculées, de S2.

**Question 7**
> Compléter le code suivant qui définit la fonction ```state```.

In [ ]:
def state(x,xp,etat):
        if etat==1 and xp>v0:
            return ####
        elif ####### and ##### :
            return 1
        else : #########

### Résolution en cas d'adhérence

Lorque S2 est en état d'adhérence, il faut résoudre :
$$\frac{dx}{dt}=v_0$$

**Question 8**
> Discrétiser cette équation pour donner la relation entre ```l_x[i+1]```, ```l_x[i]``` et les autres paramètres du problème, lors de l'état d'adhérence.

### Résolution en cas de glissement

Lors du glissement de S2, il faut résoudre l'équation d'ordre 2 :

$$m \frac{d^2x}{dt^2}=- kx+f_d mg$$

Pour résoudre ce type d'équation avec la méthode d'Euler, on va se ramener à une équation différentielle d'ordre 1, en posant le vecteur :
$$Y= \begin{bmatrix} x \\ \dot{x}\end{bmatrix}$$

**Question 9**
> Montrer que $Y$ suit une équation différentielle d'ordre 1 de type :
> $$\dot{Y} = F(Y)$$ 
> où $ F : \mathbb{R}^2 \rightarrow \mathbb{R}^2$ est une fonction dont on donnera l'expression $F(u,v)$ et dont l'on complètera la définition dans le code ci-dessous.

In [ ]:
def F(u,v):
    return ###

### Résolution entière

**Question 10**
> Compléter le code ci-dessous pour effectuer la résolution du problème et l'affichage de la position et la vitesse au cours du temps. A quoi correspondent les trais horizontaux présents dans les deux tracés ?

In [ ]:
l_etat = [etat]

while t[-1]<tf :    
    etat=state(l_x[-1],l_xp[-1],etat)
    l_etat.append(etat)
    if etat==0 :
        l_x.append(###)
        l_xp.append(###)
    else :
        a,b=F(l_x[-1],l_xp[-1])
        l_x.append(###)
        l_xp.append(###)
    
    t.append(###)
        
plt.close()
plt.figure()
plt.subplot(311)
plt.plot(t,l_x,t,[fs*m*g/k]*len(t))
plt.xlabel("Temps en secondes")
plt.ylabel("Position x en m")    
plt.grid()
plt.subplot(312)
plt.plot(t,l_xp,t,[v0]*len(t))
plt.xlabel("Temps en secondes")
plt.ylabel("Vitesse de S2 en m/s")
plt.grid()
plt.subplot(313)
plt.plot(t,l_etat)
plt.xlabel("Temps en secondes")
plt.ylabel("Etat de S2")

plt.grid()
plt.show()

### Analyse des résultats

A l'aide du code suivant, on peut déterminer le spectre du signal $x(t)$.

**Question 11**
> Exécuter le code et expliquer avec précision ce qu'il fait.


In [ ]:
t_x=np.array(l_x)
n=t_x.size

spectre=np.fft.fft(t_x) 
freq=np.fft.fftfreq(n,d=dt) 

plt.close()
plt.figure()
plt.plot(freq[0:int(len(freq)/100)],np.abs(spectre[0:int(len(freq)/100)]))

plt.xlabel("Fréquence en Hz")
plt.ylabel("Spectre en Amplitude")
plt.grid()
plt.show()

In [ ]:
max_spec=np.max(np.abs(spectre[1:int(len(freq)/100)]))
for i in range(1,int(len(freq)/100)):
    if np.abs(spectre)[i]==max_spec:
        imax=i
            
f=freq[imax]
print("La fréquence du stick-slip est",str(f)[0:4],"Hz")

## Bibliographie

Pour approfondir la théorie derrière le stick-slip, ainsi que les méthodes de résolution analytique, c'est-à-dire sans recours à la résolution numérique, on pourra consulter :

* *Exercice 5 : Mouvement « fixe-glisse » - Feuille de TD Méca 1
* *Cours 1 : crissement de la craie, mirages, cirage de bottes, phares et brouillard,...* - Claude Loverdo - http://www.lix.polytechnique.fr/Labo/Vincent.Pilaud/enseignement/cours_premiere/physique1.pdf